# 프로젝트 - Movielens 영화 추천 실습

이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.

이번에 활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. 

MovieLens 1M Dataset 사용을 권장합니다.

별점 데이터는 대표적인 explicit 데이터입니다. 

하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.

별점을 시청횟수로 해석해서 생각하겠습니다.

또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

Cloud Storage에 미리 업로드된 ml-1m폴더 내 파일을 심볼릭 링크로 개인 storage에 연결해 줍니다.

Cloud shell에서 아래 명령어를 입력해 주세요.

$ mkdir -p ~/aiffel/recommendata_iu/data/ml-1m

$ ln -s ~/data/ml-1m/* ~/aiffel/recommendata_iu/data/ml-1m

## 1) 데이터 준비와 전처리

In [26]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [27]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [28]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [29]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [30]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head(10)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [31]:
movies['movie_id_new'] = range(1,3884)
movies = movies[['movie_id', 'movie_id_new', 'title', 'genre']]
movies

,movie_id,movie_id_new,title,genre
0,1,1,Toy Story (1995),Animation|Children's|Comedy
1,2,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,3,Grumpier Old Men (1995),Comedy|Romance
3,4,4,Waiting to Exhale (1995),Comedy|Drama
4,5,5,Father of the Bride Part II (1995),Comedy
...,...,...,...,...
3878,3948,3879,Meet the Parents (2000),Comedy
3879,3949,3880,Requiem for a Dream (2000),Drama
3880,3950,3881,Tigerland (2000),Drama
3881,3951,3882,Two Family House (2000),Drama


## 2) 분석해 봅시다.
ratings에 있는 유니크한 영화 개수

ratings에 있는 유니크한 사용자 수

가장 인기 있는 영화 30개(인기순)

In [32]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [33]:
# ratings에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [34]:
# 가장 인기 있는 영화 30개(인기순)
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [35]:
movies[movies['movie_id'] == 457]

,movie_id,movie_id_new,title,genre
453,457,454,"Fugitive, The (1993)",Action|Thriller


In [36]:
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

In [37]:
data = pd.merge(ratings, movies, on='movie_id', how='left')
# data = data[['user_id', 'movie_id', 'movie_id_new', 'counts', 'title', 'genre']] # 이렇게 맞춰야 데이터가 나온다 아니면 Nan으로 나옴
data = data[['user_id', 'movie_id', 'counts', 'title', 'genre']]

data.head(10)

,user_id,movie_id,counts,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,My Fair Lady (1964),Musical|Romance
3,1,3408,4,Erin Brockovich (2000),Drama
4,1,2355,5,"Bug's Life, A (1998)",Animation|Children's|Comedy
5,1,1197,3,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance
6,1,1287,5,Ben-Hur (1959),Action|Adventure|Drama
7,1,2804,5,"Christmas Story, A (1983)",Comedy|Drama
8,1,594,4,Snow White and the Seven Dwarfs (1937),Animation|Children's|Musical
9,1,919,4,"Wizard of Oz, The (1939)",Adventure|Children's|Drama|Musical


In [38]:
data[data['movie_id'] == 1]

,user_id,movie_id,counts,title,genre
40,1,1,5,Toy Story (1995),Animation|Children's|Comedy
393,6,1,4,Toy Story (1995),Animation|Children's|Comedy
503,8,1,4,Toy Story (1995),Animation|Children's|Comedy
626,9,1,5,Toy Story (1995),Animation|Children's|Comedy
743,10,1,5,Toy Story (1995),Animation|Children's|Comedy
...,...,...,...,...,...
834064,6022,1,5,Toy Story (1995),Animation|Children's|Comedy
834336,6025,1,5,Toy Story (1995),Animation|Children's|Comedy
834859,6032,1,4,Toy Story (1995),Animation|Children's|Comedy
835030,6035,1,4,Toy Story (1995),Animation|Children's|Comedy


## 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [39]:
movies[movies['movie_id'] == 1]

,movie_id,movie_id_new,title,genre
0,1,1,Toy Story (1995),Animation|Children's|Comedy


In [40]:
movies[movies['movie_id'] == 2]

,movie_id,movie_id_new,title,genre
1,2,2,Jumanji (1995),Adventure|Children's|Fantasy


In [41]:
movies[movies['movie_id'] == 356]

,movie_id,movie_id_new,title,genre
352,356,353,Forrest Gump (1994),Comedy|Romance|War


In [42]:
movies[movies['movie_id'] == 480]

,movie_id,movie_id_new,title,genre
476,480,477,Jurassic Park (1993),Action|Adventure|Sci-Fi


In [43]:
movies[movies['movie_id'] == 2762]

,movie_id,movie_id_new,title,genre
2693,2762,2694,"Sixth Sense, The (1999)",Thriller


In [44]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ["1" , "2" ,"356" ,"480" ,"2762"]
my_title = ["Toy Story (1995)", "Jumanji (1995)", "Forrest Gump (1994)", "Jurassic Park (1993)", "Sixth Sense, The (1999)" ]
my_genre = ["Animation|Children's|Comedy", "Adventure|Children's|Fantasy", "Comedy|Romance|War", "Action|Adventure|Sci-Fi", "Thriller"]

# 'ikhan'이라는 user_id가 위 영화의 폄점을 5점씩 줬다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['ikhan']*5, 'movie_id': my_favorite, 'counts':[5]*5, 'title': my_title, 'genre': my_genre})

if not data.isin({'user_id':['ikhan']})['user_id'].any():  # user_id에 'ikhan'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts,title,genre
836473,6040,1090,3,Platoon (1986),Drama|War
836474,6040,1094,5,"Crying Game, The (1992)",Drama|Romance|War
836475,6040,562,5,Welcome to the Dollhouse (1995),Comedy|Drama
836476,6040,1096,4,Sophie's Choice (1982),Drama
836477,6040,1097,4,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
0,ikhan,1,5,Toy Story (1995),Animation|Children's|Comedy
1,ikhan,2,5,Jumanji (1995),Adventure|Children's|Fantasy
2,ikhan,356,5,Forrest Gump (1994),Comedy|Romance|War
3,ikhan,480,5,Jurassic Park (1993),Action|Adventure|Sci-Fi
4,ikhan,2762,5,"Sixth Sense, The (1999)",Thriller


In [45]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [46]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['ikhan'])
print(title_to_idx['Jumanji (1995)'])

6039
513


In [47]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# title_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('titlet column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
titlet column indexing OK!!


,user_id,movie_id,counts,title,genre
0,0,1193,5,0,Drama
1,0,661,3,1,Animation|Children's|Musical
2,0,914,3,2,Musical|Romance
3,0,3408,4,3,Drama
4,0,2355,5,4,Animation|Children's|Comedy
...,...,...,...,...,...
0,6039,1,5,40,Animation|Children's|Comedy
1,6039,2,5,513,Adventure|Children's|Fantasy
2,6039,356,5,160,Comedy|Romance|War
3,6039,480,5,107,Action|Adventure|Sci-Fi


## 4) CSR matrix를 직접 만들어 봅시다.

In [49]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

csr_data = csr_matrix((data['counts'], (data.user_id, data.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [50]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [51]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, 
                                    iterations=15, dtype = np.float32)

In [52]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [53]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

### 선호하는 5가지 영화 중 하나

In [59]:
ikhan, jumanji = user_to_idx['ikhan'], title_to_idx['Jumanji (1995)']
ikhan_vector, jumanji_vector = als_model.user_factors[ikhan], als_model.item_factors[jumanji]

In [60]:
ikhan_vector

array([ 1.0537133 , -0.35998052, -0.8011585 ,  0.3951567 , -0.30624768,
        1.169707  , -0.11644905, -0.25276878,  0.73340803,  0.08640683,
       -0.28619584, -0.11455519,  0.30001935, -0.12252838, -0.42721295,
        0.10871723,  0.15289101, -0.3182896 ,  0.64717835,  0.2512372 ,
       -0.25177667, -0.6407526 ,  1.074436  ,  0.42192122,  0.1777401 ,
        0.08900782,  0.22046478, -0.3784971 , -0.775128  , -0.4506531 ,
       -0.8202472 ,  0.0695649 ,  0.7321117 ,  0.44583166, -0.4276113 ,
       -0.07786975, -0.5585579 , -0.04429226,  1.1305456 ,  0.564292  ,
        0.7526316 ,  0.15527482,  0.5601096 ,  0.07380766, -0.4951604 ,
        0.25021964, -1.0251001 ,  0.7020633 ,  0.14638521, -0.12076724,
        0.24854553, -0.7116092 , -0.00773837, -0.07331428, -0.28632307,
        0.1863223 ,  0.37204713, -1.2247711 , -0.67852694, -0.3208242 ,
       -0.62302905,  0.36237755, -0.6365195 , -0.36816475, -0.48620993,
        0.49402636, -0.9316712 ,  0.23538637,  0.19601299,  0.86

In [62]:
jumanji_vector

array([ 2.08583623e-02,  4.21512313e-03, -5.46714012e-03,  2.22948398e-02,
        1.81884297e-05,  4.05504443e-02,  3.03869601e-03,  2.33607609e-02,
        2.03484148e-02,  6.45654509e-03,  7.22477818e-03,  2.60840580e-02,
        3.15237269e-02, -1.76334772e-02, -3.36319068e-03, -8.76403181e-04,
        5.97143022e-04,  4.80415160e-03,  6.60090055e-03, -2.12040660e-03,
        2.24950798e-02,  5.42800175e-03,  3.39126177e-02,  3.03810071e-02,
        3.37459594e-02,  3.23087838e-03, -8.66791047e-03, -2.15635332e-03,
       -1.18161486e-02, -1.26261879e-02, -5.31062391e-03, -2.88250651e-02,
        5.14358655e-03,  3.03121842e-03, -1.14755547e-02,  8.59544729e-04,
        4.54313448e-03,  2.42721085e-02,  1.12007251e-02,  1.59843657e-02,
       -9.64311417e-03,  2.33076848e-02,  1.86708458e-02, -9.33913235e-03,
        2.05354975e-03,  1.28417760e-02, -1.45287458e-02,  4.95533273e-03,
        2.95625231e-03,  9.03096888e-03,  1.80400442e-02,  1.38361705e-03,
       -1.42633412e-02,  

In [63]:
# ikhan와 jumanji을 내적하는 코드
np.set_printoptions(formatter={'float_kind': lambda x: "{0:0.3f}".format(x)})

print(np.dot(ikhan_vector, jumanji_vector))

0.27225986


### 그 외의 영화

In [64]:
print(title_to_idx['Ben-Hur (1959)'])

6


In [66]:
ikhan, benhur = user_to_idx['ikhan'], title_to_idx['Ben-Hur (1959)']
ikhan_vector, benhur_vector = als_model.user_factors[ikhan], als_model.item_factors[benhur]

In [67]:
ikhan_vector

array([1.054, -0.360, -0.801, 0.395, -0.306, 1.170, -0.116, -0.253, 0.733,
       0.086, -0.286, -0.115, 0.300, -0.123, -0.427, 0.109, 0.153, -0.318,
       0.647, 0.251, -0.252, -0.641, 1.074, 0.422, 0.178, 0.089, 0.220,
       -0.378, -0.775, -0.451, -0.820, 0.070, 0.732, 0.446, -0.428,
       -0.078, -0.559, -0.044, 1.131, 0.564, 0.753, 0.155, 0.560, 0.074,
       -0.495, 0.250, -1.025, 0.702, 0.146, -0.121, 0.249, -0.712, -0.008,
       -0.073, -0.286, 0.186, 0.372, -1.225, -0.679, -0.321, -0.623,
       0.362, -0.637, -0.368, -0.486, 0.494, -0.932, 0.235, 0.196, 0.866,
       -0.110, 0.493, -0.348, -0.404, 0.779, -1.045, 0.508, 0.110, 0.323,
       0.715, -0.625, 0.399, -0.121, -0.261, 0.170, -0.873, 0.728, 0.185,
       0.834, 0.276, 0.565, 0.441, -0.490, 0.338, -0.398, 0.181, -0.768,
       -0.118, 0.521, -0.113], dtype=float32)

In [68]:
benhur_vector

array([-0.004, 0.041, -0.005, 0.002, -0.029, 0.019, 0.009, 0.007, 0.007,
       -0.003, -0.019, 0.004, -0.007, 0.024, -0.012, 0.023, 0.009, -0.007,
       0.009, -0.010, 0.015, -0.015, 0.022, 0.023, -0.019, -0.003, 0.010,
       -0.012, 0.010, 0.032, -0.033, -0.011, 0.028, 0.001, 0.025, -0.002,
       0.017, -0.001, 0.013, 0.003, -0.016, 0.021, -0.014, 0.019, 0.027,
       -0.000, 0.013, 0.005, -0.004, 0.009, 0.006, 0.022, 0.019, -0.012,
       0.011, 0.022, 0.007, 0.010, 0.015, 0.037, 0.000, 0.023, -0.012,
       0.018, 0.003, 0.013, 0.028, 0.031, 0.006, -0.009, -0.022, 0.025,
       -0.002, 0.010, 0.004, 0.015, -0.011, -0.004, 0.021, -0.009, -0.003,
       0.026, 0.003, -0.001, 0.015, 0.028, 0.015, -0.005, 0.021, -0.010,
       0.014, -0.023, 0.002, 0.015, 0.013, 0.002, 0.033, -0.003, 0.007,
       0.002], dtype=float32)

In [69]:
print(np.dot(ikhan_vector, benhur_vector))

-0.014070309


## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [70]:
favorite_movie = 'Jumanji (1995)'
movie_id =  title_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(513, 1.0000001),
 (1130, 0.79469424),
 (596, 0.7836685),
 (173, 0.76271784),
 (545, 0.6786715),
 (828, 0.6616487),
 (1733, 0.64524144),
 (1985, 0.6438804),
 (1982, 0.6226522),
 (1736, 0.60915893),
 (561, 0.5990693),
 (458, 0.5743479),
 (576, 0.5726645),
 (611, 0.5656132),
 (942, 0.5422552)]

In [71]:
#title_to_idx 를 뒤집어, index로부터 title 이름을 얻는 dict를 생성합니다. 
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_movie]

['Jumanji (1995)',
 'Indian in the Cupboard, The (1995)',
 'Hook (1991)',
 'Dragonheart (1996)',
 'Santa Clause, The (1994)',
 'Flubber (1997)',
 'Borrowers, The (1997)',
 'NeverEnding Story II: The Next Chapter, The (1990)',
 'Space Jam (1996)',
 'Small Soldiers (1998)',
 'NeverEnding Story, The (1984)',
 'Mask, The (1994)',
 'Escape to Witch Mountain (1975)',
 'Labyrinth (1986)',
 'Legend (1985)']

In [72]:
def get_similar_movie(movie_name: str):
    movie_id = title_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_title[i[0]] for i in similar_movie]
    return similar_movie

In [73]:
get_similar_movie('Forrest Gump (1994)')

['Forrest Gump (1994)',
 'Groundhog Day (1993)',
 'Pretty Woman (1990)',
 'Sleepless in Seattle (1993)',
 'Ghost (1990)',
 'As Good As It Gets (1997)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Pleasantville (1998)',
 'Four Weddings and a Funeral (1994)',
 'Clueless (1995)']

In [74]:
get_similar_movie('Sixth Sense, The (1999)')

['Sixth Sense, The (1999)',
 'Usual Suspects, The (1995)',
 'Silence of the Lambs, The (1991)',
 'Fight Club (1999)',
 'Ghostbusters (1984)',
 'Seven (Se7en) (1995)',
 'Matrix, The (1999)',
 'Green Mile, The (1999)',
 'Sleepy Hollow (1999)',
 'Fugitive, The (1993)']

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [75]:
user = user_to_idx['ikhan']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(50, 0.46562594),
 (124, 0.39316624),
 (175, 0.3724543),
 (92, 0.3720152),
 (110, 0.3568868),
 (64, 0.31433624),
 (60, 0.3075305),
 (141, 0.2893923),
 (458, 0.28584635),
 (62, 0.25389028),
 (670, 0.23601954),
 (58, 0.23300639),
 (330, 0.23186184),
 (99, 0.22744444),
 (121, 0.22625875),
 (4, 0.22113982),
 (33, 0.21544234),
 (322, 0.21502908),
 (48, 0.21256728),
 (255, 0.20526429)]

In [77]:
rec = [idx_to_title[i[0]] for i in movie_recommended]
recommendation = pd.DataFrame(rec)
recommendation.columns = ['추천 영화 Top 20']
recommendation

,추천 영화 Top 20
0,Toy Story 2 (1999)
1,"Matrix, The (1999)"
2,Men in Black (1997)
3,Terminator 2: Judgment Day (1991)
4,Groundhog Day (1993)
5,Star Wars: Episode VI - Return of the Jedi (1983)
6,Star Wars: Episode I - The Phantom Menace (1999)
7,"Fugitive, The (1993)"
8,"Mask, The (1994)"
9,Total Recall (1990)


In [78]:
mask = title_to_idx['Mask, The (1994)']
explain = als_model.explain(user, csr_data, itemid=mask)

In [79]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('Jumanji (1995)', 0.15382299567867663),
 ('Toy Story (1995)', 0.062188179614400374),
 ('Forrest Gump (1994)', 0.044734911695568196),
 ('Sixth Sense, The (1999)', 0.013483401480204068),
 ('Jurassic Park (1993)', 0.008583437837593836)]

# 회고

노드에서 했던것들 위주로 따라서 했는데 잘 진행되었고 선호하는 작품 하나로 선호도 예측 하는 부분은 평가한게 몇작품 없어서 그런지 좀 낮게 나오긴 했지만 내가 좋아하는 영화와 비슷한 영화를 추천 하는부분에는 만족으러운 영화들이 추천되었고 내가 가장 좋아할 만한 영화들은 추천 받는 부분에서도 취향을 제대로 파악하여 추천 해주었음 5가지가 아니라 더 많은 영화에 평가를 한후 선호도 예측을 하면 더욱더 좋게 나올것으로 예상된다. 이번 프로젝트는 과정도 그렇고 프로젝트에서 다룬 영화라는 취향도 그렇고 결과에서 나온 부분을 봤을때 재밌는 주제와 프로젝트였다